In [1]:
import pandas as pd

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [2]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
from sklearn.base import BaseEstimator, TransformerMixin

# A class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer

imputer = Imputer(strategy="median")

num_pipeline = Pipeline([
        ("select_numeric", DataFrameSelector(["Age", "SibSp", "Parch", "Fare"])),
        ("imputer", Imputer(strategy="median")),
    ])

In [5]:
num_pipeline.fit_transform(train_data)

array([[22.    ,  1.    ,  0.    ,  7.25  ],
       [38.    ,  1.    ,  0.    , 71.2833],
       [26.    ,  0.    ,  0.    ,  7.925 ],
       ...,
       [28.    ,  1.    ,  2.    , 23.45  ],
       [26.    ,  0.    ,  0.    , 30.    ],
       [32.    ,  0.    ,  0.    ,  7.75  ]])

In [6]:
# We also need an imputer for the string categorical columns
# (The regular imputer does not work on those)
class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X],
                                       index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

In [7]:
from future_encoders import OneHotEncoder

In [8]:
cat_pipeline = Pipeline([
    ('select_cat', DataFrameSelector(['Pclass', 'Sex', 'Embarked'])),
    ('imputer', MostFrequentImputer()),
    ('cat_encoder', OneHotEncoder(sparse=False))
])

In [9]:
from sklearn.pipeline import FeatureUnion
preprocess_pipeline = FeatureUnion(transformer_list=[
    ('num_pipeline', num_pipeline),
    ('cat_pipeline', cat_pipeline),
])

In [10]:
X_train = preprocess_pipeline.fit_transform(train_data)
y_train = train_data['Survived']
X_test = preprocess_pipeline.transform(test_data)

In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [12]:
X_train.shape
accuracy = []

In [14]:
import tensorflow

tf.reset_default_graph()
n_inputs = X_train.shape[1] #12
n_hidden1 = 25
n_hidden2 = 15
n_outputs = 2

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int64, shape=(None), name='y')

In [19]:
from tensorflow.contrib.layers import fully_connected
from tensorflow.contrib.layers import dropout

is_training = tf.placeholder_with_default(False, shape=(), name='is_training')
keep_prob = 0.5
X_drop = dropout(X, keep_prob, is_training=is_training)

with tf.name_scope('dnn'):
    hidden1 = fully_connected(X, n_hidden1, scope='hidden1')
    hidden1_drop = dropout(hidden1, keep_prob, is_training=is_training)
    
    hidden2 = fully_connected(hidden1_drop, n_hidden2, scope='hidden2')
    hidden2_drop = dropout(hidden2, keep_prob, is_training=is_training)
    
    logits = fully_connected(hidden2_drop, n_outputs, scope='outputs', activation_fn=None)

In [20]:
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name='loss')
# softmax_cross_entropy_with_logits can handle one-hot encoding, while sparse_softmax_cross_entropy_with_loss handles
# ints from 0 to the number of classes-1

In [21]:
learning_rate = 0.001

with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [22]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [23]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [24]:
import numpy as np

def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [25]:
n_epochs = 5000
batch_size = 30

In [26]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        if epoch % 50 == 0: acc_total = accuracy.eval(feed_dict={X: X_train, y: y_train})
        print(epoch, "Batch accuracy:", acc_batch, "Train accuracy", acc_total)

    save_path = saver.save(sess, "./adam_model.ckpt")
    Z = logits.eval(feed_dict={X: X_test})
    y_preds = np.argmax(Z, axis=1)

0 Batch accuracy: 0.46666667 Train accuracy 0.56453425
1 Batch accuracy: 0.73333335 Train accuracy 0.56453425
2 Batch accuracy: 0.53333336 Train accuracy 0.56453425
3 Batch accuracy: 0.6666667 Train accuracy 0.56453425
4 Batch accuracy: 0.76666665 Train accuracy 0.56453425
5 Batch accuracy: 0.6666667 Train accuracy 0.56453425
6 Batch accuracy: 0.7 Train accuracy 0.56453425
7 Batch accuracy: 0.76666665 Train accuracy 0.56453425
8 Batch accuracy: 0.73333335 Train accuracy 0.56453425
9 Batch accuracy: 0.7 Train accuracy 0.56453425
10 Batch accuracy: 0.73333335 Train accuracy 0.56453425
11 Batch accuracy: 0.73333335 Train accuracy 0.56453425
12 Batch accuracy: 0.8333333 Train accuracy 0.56453425
13 Batch accuracy: 0.8333333 Train accuracy 0.56453425
14 Batch accuracy: 0.93333334 Train accuracy 0.56453425
15 Batch accuracy: 0.73333335 Train accuracy 0.56453425
16 Batch accuracy: 0.8666667 Train accuracy 0.56453425
17 Batch accuracy: 0.7 Train accuracy 0.56453425
18 Batch accuracy: 0.7333333

158 Batch accuracy: 0.8333333 Train accuracy 0.8327722
159 Batch accuracy: 0.8 Train accuracy 0.8327722
160 Batch accuracy: 0.76666665 Train accuracy 0.8327722
161 Batch accuracy: 0.8 Train accuracy 0.8327722
162 Batch accuracy: 0.8666667 Train accuracy 0.8327722
163 Batch accuracy: 0.8333333 Train accuracy 0.8327722
164 Batch accuracy: 0.6666667 Train accuracy 0.8327722
165 Batch accuracy: 0.76666665 Train accuracy 0.8327722
166 Batch accuracy: 0.7 Train accuracy 0.8327722
167 Batch accuracy: 0.8333333 Train accuracy 0.8327722
168 Batch accuracy: 0.8333333 Train accuracy 0.8327722
169 Batch accuracy: 0.6666667 Train accuracy 0.8327722
170 Batch accuracy: 0.93333334 Train accuracy 0.8327722
171 Batch accuracy: 0.76666665 Train accuracy 0.8327722
172 Batch accuracy: 0.8 Train accuracy 0.8327722
173 Batch accuracy: 0.8666667 Train accuracy 0.8327722
174 Batch accuracy: 0.8333333 Train accuracy 0.8327722
175 Batch accuracy: 0.8666667 Train accuracy 0.8327722
176 Batch accuracy: 0.8333333 

320 Batch accuracy: 0.8 Train accuracy 0.82828283
321 Batch accuracy: 0.76666665 Train accuracy 0.82828283
322 Batch accuracy: 0.8 Train accuracy 0.82828283
323 Batch accuracy: 0.8666667 Train accuracy 0.82828283
324 Batch accuracy: 0.9 Train accuracy 0.82828283
325 Batch accuracy: 0.8666667 Train accuracy 0.82828283
326 Batch accuracy: 0.8666667 Train accuracy 0.82828283
327 Batch accuracy: 0.76666665 Train accuracy 0.82828283
328 Batch accuracy: 0.8333333 Train accuracy 0.82828283
329 Batch accuracy: 0.8333333 Train accuracy 0.82828283
330 Batch accuracy: 0.7 Train accuracy 0.82828283
331 Batch accuracy: 0.6333333 Train accuracy 0.82828283
332 Batch accuracy: 0.8333333 Train accuracy 0.82828283
333 Batch accuracy: 0.73333335 Train accuracy 0.82828283
334 Batch accuracy: 0.8666667 Train accuracy 0.82828283
335 Batch accuracy: 0.8333333 Train accuracy 0.82828283
336 Batch accuracy: 0.76666665 Train accuracy 0.82828283
337 Batch accuracy: 0.96666664 Train accuracy 0.82828283
338 Batch a

482 Batch accuracy: 0.76666665 Train accuracy 0.8507295
483 Batch accuracy: 0.8666667 Train accuracy 0.8507295
484 Batch accuracy: 0.8666667 Train accuracy 0.8507295
485 Batch accuracy: 0.8 Train accuracy 0.8507295
486 Batch accuracy: 0.8333333 Train accuracy 0.8507295
487 Batch accuracy: 0.8 Train accuracy 0.8507295
488 Batch accuracy: 0.8333333 Train accuracy 0.8507295
489 Batch accuracy: 0.8666667 Train accuracy 0.8507295
490 Batch accuracy: 0.76666665 Train accuracy 0.8507295
491 Batch accuracy: 0.8 Train accuracy 0.8507295
492 Batch accuracy: 0.9 Train accuracy 0.8507295
493 Batch accuracy: 0.93333334 Train accuracy 0.8507295
494 Batch accuracy: 0.8333333 Train accuracy 0.8507295
495 Batch accuracy: 0.76666665 Train accuracy 0.8507295
496 Batch accuracy: 0.76666665 Train accuracy 0.8507295
497 Batch accuracy: 0.8333333 Train accuracy 0.8507295
498 Batch accuracy: 0.9 Train accuracy 0.8507295
499 Batch accuracy: 0.9 Train accuracy 0.8507295
500 Batch accuracy: 0.93333334 Train accu

635 Batch accuracy: 0.93333334 Train accuracy 0.83613914
636 Batch accuracy: 0.9 Train accuracy 0.83613914
637 Batch accuracy: 0.9 Train accuracy 0.83613914
638 Batch accuracy: 0.8333333 Train accuracy 0.83613914
639 Batch accuracy: 0.9 Train accuracy 0.83613914
640 Batch accuracy: 0.8 Train accuracy 0.83613914
641 Batch accuracy: 0.9 Train accuracy 0.83613914
642 Batch accuracy: 0.8666667 Train accuracy 0.83613914
643 Batch accuracy: 0.8 Train accuracy 0.83613914
644 Batch accuracy: 0.76666665 Train accuracy 0.83613914
645 Batch accuracy: 0.8666667 Train accuracy 0.83613914
646 Batch accuracy: 0.73333335 Train accuracy 0.83613914
647 Batch accuracy: 0.8666667 Train accuracy 0.83613914
648 Batch accuracy: 0.8 Train accuracy 0.83613914
649 Batch accuracy: 0.8333333 Train accuracy 0.83613914
650 Batch accuracy: 0.9 Train accuracy 0.85746354
651 Batch accuracy: 0.8 Train accuracy 0.85746354
652 Batch accuracy: 0.8333333 Train accuracy 0.85746354
653 Batch accuracy: 0.8 Train accuracy 0.85

...

4293 Batch accuracy: 1.0 Train accuracy 0.9304153
4294 Batch accuracy: 0.96666664 Train accuracy 0.9304153
4295 Batch accuracy: 0.9 Train accuracy 0.9304153
4296 Batch accuracy: 0.9 Train accuracy 0.9304153
4297 Batch accuracy: 0.93333334 Train accuracy 0.9304153
4298 Batch accuracy: 0.93333334 Train accuracy 0.9304153
4299 Batch accuracy: 0.93333334 Train accuracy 0.9304153
4300 Batch accuracy: 0.93333334 Train accuracy 0.9214366
4301 Batch accuracy: 0.93333334 Train accuracy 0.9214366
4302 Batch accuracy: 0.93333334 Train accuracy 0.9214366
4303 Batch accuracy: 0.8666667 Train accuracy 0.9214366
4304 Batch accuracy: 1.0 Train accuracy 0.9214366
4305 Batch accuracy: 0.96666664 Train accuracy 0.9214366
4306 Batch accuracy: 0.8666667 Train accuracy 0.9214366
4307 Batch accuracy: 0.93333334 Train accuracy 0.9214366
4308 Batch accuracy: 0.9 Train accuracy 0.9214366
4309 Batch accuracy: 0.8666667 Train accuracy 0.9214366
4310 Batch accuracy: 0.93333334 Train accuracy 0.9214366
4311 Batch a

4446 Batch accuracy: 1.0 Train accuracy 0.9292929
4447 Batch accuracy: 0.93333334 Train accuracy 0.9292929
4448 Batch accuracy: 0.9 Train accuracy 0.9292929
4449 Batch accuracy: 0.9 Train accuracy 0.9292929
4450 Batch accuracy: 0.96666664 Train accuracy 0.92368126
4451 Batch accuracy: 0.96666664 Train accuracy 0.92368126
4452 Batch accuracy: 0.96666664 Train accuracy 0.92368126
4453 Batch accuracy: 0.93333334 Train accuracy 0.92368126
4454 Batch accuracy: 0.8666667 Train accuracy 0.92368126
4455 Batch accuracy: 0.96666664 Train accuracy 0.92368126
4456 Batch accuracy: 0.8333333 Train accuracy 0.92368126
4457 Batch accuracy: 0.96666664 Train accuracy 0.92368126
4458 Batch accuracy: 1.0 Train accuracy 0.92368126
4459 Batch accuracy: 0.9 Train accuracy 0.92368126
4460 Batch accuracy: 0.9 Train accuracy 0.92368126
4461 Batch accuracy: 0.8666667 Train accuracy 0.92368126
4462 Batch accuracy: 0.96666664 Train accuracy 0.92368126
4463 Batch accuracy: 0.93333334 Train accuracy 0.92368126
4464 

4600 Batch accuracy: 0.96666664 Train accuracy 0.9248036
4601 Batch accuracy: 0.9 Train accuracy 0.9248036
4602 Batch accuracy: 0.9 Train accuracy 0.9248036
4603 Batch accuracy: 0.96666664 Train accuracy 0.9248036
4604 Batch accuracy: 0.93333334 Train accuracy 0.9248036
4605 Batch accuracy: 0.9 Train accuracy 0.9248036
4606 Batch accuracy: 0.93333334 Train accuracy 0.9248036
4607 Batch accuracy: 0.9 Train accuracy 0.9248036
4608 Batch accuracy: 0.93333334 Train accuracy 0.9248036
4609 Batch accuracy: 0.9 Train accuracy 0.9248036
4610 Batch accuracy: 1.0 Train accuracy 0.9248036
4611 Batch accuracy: 0.93333334 Train accuracy 0.9248036
4612 Batch accuracy: 0.8666667 Train accuracy 0.9248036
4613 Batch accuracy: 0.93333334 Train accuracy 0.9248036
4614 Batch accuracy: 0.96666664 Train accuracy 0.9248036
4615 Batch accuracy: 1.0 Train accuracy 0.9248036
4616 Batch accuracy: 0.9 Train accuracy 0.9248036
4617 Batch accuracy: 0.8666667 Train accuracy 0.9248036
4618 Batch accuracy: 0.8333333 T

4753 Batch accuracy: 0.96666664 Train accuracy 0.92704827
4754 Batch accuracy: 0.8333333 Train accuracy 0.92704827
4755 Batch accuracy: 0.9 Train accuracy 0.92704827
4756 Batch accuracy: 0.96666664 Train accuracy 0.92704827
4757 Batch accuracy: 0.9 Train accuracy 0.92704827
4758 Batch accuracy: 0.96666664 Train accuracy 0.92704827
4759 Batch accuracy: 0.8666667 Train accuracy 0.92704827
4760 Batch accuracy: 0.8666667 Train accuracy 0.92704827
4761 Batch accuracy: 0.9 Train accuracy 0.92704827
4762 Batch accuracy: 1.0 Train accuracy 0.92704827
4763 Batch accuracy: 0.96666664 Train accuracy 0.92704827
4764 Batch accuracy: 0.8666667 Train accuracy 0.92704827
4765 Batch accuracy: 0.93333334 Train accuracy 0.92704827
4766 Batch accuracy: 0.93333334 Train accuracy 0.92704827
4767 Batch accuracy: 0.9 Train accuracy 0.92704827
4768 Batch accuracy: 0.96666664 Train accuracy 0.92704827
4769 Batch accuracy: 1.0 Train accuracy 0.92704827
4770 Batch accuracy: 0.96666664 Train accuracy 0.92704827
47

4910 Batch accuracy: 0.96666664 Train accuracy 0.9259259
4911 Batch accuracy: 0.9 Train accuracy 0.9259259
4912 Batch accuracy: 0.9 Train accuracy 0.9259259
4913 Batch accuracy: 1.0 Train accuracy 0.9259259
4914 Batch accuracy: 0.9 Train accuracy 0.9259259
4915 Batch accuracy: 0.93333334 Train accuracy 0.9259259
4916 Batch accuracy: 0.96666664 Train accuracy 0.9259259
4917 Batch accuracy: 0.9 Train accuracy 0.9259259
4918 Batch accuracy: 0.93333334 Train accuracy 0.9259259
4919 Batch accuracy: 0.9 Train accuracy 0.9259259
4920 Batch accuracy: 0.9 Train accuracy 0.9259259
4921 Batch accuracy: 1.0 Train accuracy 0.9259259
4922 Batch accuracy: 0.93333334 Train accuracy 0.9259259
4923 Batch accuracy: 0.96666664 Train accuracy 0.9259259
4924 Batch accuracy: 0.8666667 Train accuracy 0.9259259
4925 Batch accuracy: 0.8333333 Train accuracy 0.9259259
4926 Batch accuracy: 0.8 Train accuracy 0.9259259
4927 Batch accuracy: 0.96666664 Train accuracy 0.9259259
4928 Batch accuracy: 0.93333334 Train a

In [27]:
with tf.Session() as sess:
    saver.restore(sess, "./adam_model.ckpt")
    Z = logits.eval(feed_dict={X: X_test})
    y_predictions = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./adam_model.ckpt


In [28]:
d = {'PassengerId':[i+892 for i in range(len(list(y_predictions)))],
       'Survived': list(y_predictions)}
df = pd.DataFrame(data=d, index=None)
df.to_csv('adam_submission.csv', index=False)

In [29]:
test = pd.read_csv('titanic_test.csv')

In [30]:
x_test_new = preprocess_pipeline.fit_transform(test)
 